Decision Tree- ID3

In [1]:
import numpy as np
import pandas as pd

In [17]:
def find_entropy(df):
  target=df.keys()[-1]
  target_values=df[target].unique()
  entropy=0
  for value in target_values:
    prob=len(df[df[target]==value])/len(df)  # gives prob of each value of each class
    entropy+= -(prob*np.log2(prob)) # Done for every unique variable of target variable
  return entropy  

In [18]:
df=pd.read_csv('weather.csv')

In [19]:
find_entropy(df)

0.9402859586706309

In [22]:
from tables import attributeset
# find the entropy of every sub sample
#we need the unique value of each sub sample as well as the unique values based on the target variable
def find_avg_info_entropy(df,attribute):
  attr_values=df[attribute].unique()
  target=df.keys()[-1]
  target_values=df[target].unique()
  avg_info_entropy=0
  for value1 in attr_values:
    entropy_subsample=0
    for value2 in target_values: # Check positive and Negative
      num=len(df[df[attribute]==value1][df[target]==value2]) #AND
      den=len(df[df[attribute]==value1])
      prob=num/den
      entropy_subsample+= -(prob*np.log2(prob+1e-7))
    weight= den/len(df)  #den= number of rows with subsample
    avg_info_entropy+=weight*entropy_subsample
  return avg_info_entropy  



In [28]:
find_avg_info_entropy(df,'Outlook')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


0.6935358915770655

In [29]:
find_avg_info_entropy(df,'Humidity')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


0.7884501687693255

In [30]:
#To find whihc sample has the highest Info Gain
def find_winner(df):
  df.keys()[: -1] #We find IG of only input attributes except the last coumn which is the target
  attributes=df.keys()[: -1]
  IG=[]
  for attribute in attributes:
    IG.append(find_entropy(df)- find_avg_info_entropy(df,attribute))
  return df.keys()[: -1][np.argmax(IG)] 

In [31]:
find_winner(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


'Outlook'

In [35]:
#Constructing the Decision Tree
def training(df,tree=None):
  target=df.keys()[-1]
  attribute=find_winner(df)
  attr_values=df[attribute].unique()
  if tree is None:
    tree={}
    tree[attribute]={} #Making the first winner the root node #Now we need to iterate over the unique values
    for value in attr_values:
      sub_df=df[df[attribute]==value].reset_index(drop=True) #reset_index(): The random row number received get converted into 0,1,2..
      clvalue,count=np.unique(sub_df[target],return_counts=True) # gives 2 list: 1st: number of unique values and 2nd: Count of unique value
      if len(count)==1: #if it belongs to just one class either yes/no
        tree[attribute][value]=clvalue[0]
      else:
        tree[attribute][value]=training(sub_df)
    return tree   

          





In [40]:
tree=training(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


Outlook is the key and under outlook we have 3 attributes : overcast,rainy and sunny as overcast has just 1 class in the target variable it will become a leaf node.

In [41]:
def prediction(instance,tree):
  for attribute in tree.keys():
    value=instance[attribute]
    tree=tree[attribute][value] #If a dictionary then recursively call else return the value and break
    if type(tree) is dict:
      predict=prediction(instance,tree)
    else:
      predict=tree
      break
  return predict      

In [42]:
df1=pd.read_csv('weather_test.csv')
y_predict=[]
for i in range (len(df1)):
  instance=df1.iloc[i,:]
  predict=prediction(instance,tree)
  y_predict.append(predict)

In [43]:
y_predict

[0, 1]